# Atenção: Antes de iniciar verifique o checklist:

- Baixar todos os pacotes necessários do pip.
    `pip install numpy matplotlib scipy gdal pyproj scikit-image pyhdf pyresample netcdf4 h5py requests pygrib cartopy sentinelsat tqdm jupyter`

In [ ]:
import os, stat
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
from zipfile import ZipFile
from glob import glob
from tqdm import tqdm
from os import getcwd, makedirs
from os.path import exists, join, basename
from IPython.display import clear_output

### Todos os parâmetros necessários para o programa funcionar corretamente

In [ ]:
# CAMINHO ONDE O SENTINELSAT IRÁ ARMAZENAR OS ARQUIVOS BAIXADOS
CAMINHO_OUTPUT_DOWNLOAD = 'minha_pasta/pasta1/'
# CAMINHO ONDE SERÁ ARMAZENADO OS ARQUIVOS EXTRAIDOS BAIXADO DO SENTINELSAT
CAMINHO_OUTPUT_EXTRAIDO = 'minha_pasta/extraidos/'
# CAMINHO PARA O GEOJSON DA ÁREA DE INTERESSE
CAMINHO_AREA_ESTUDO_GEOJSON = 'minha_pasta/area_estudo.geojson'

# USUÁRIO E SENHA UTILIZADO NO EO BROWSER
USUARIO = 'meu_usuario'
SENHA = 'minha_senha'

# CAMINHO DE OUTPUT DO PROCESSAMENTO DO ACOLITE
CAMINHO_OUTPUT_PROCESSADO_ACOLITE = 'minha_pasta/processados'

### Configurações do acolite

- Para evitar qualquer trabalho extra, será usado um dicionário python para substituir o .txt de configurações do acolite assim como diz a documentação (PDF página 10 última caixa de código)

In [ ]:
acolite_configs = {
    'delete_acolite_run_text_file': True,
    'oli_orange_band': False,
    'smile_correction': False,
    'use_tpg': False,
    'l2w_parameters': 'rhow_*',
    'dsf_interface_reflectance': True,
    'dsf_residual_glint_correction': True,
    'rgb_rhot': True,
    'rgb_rhos': True,
    'map_l2w': True,
    'l2w_export_geotiff': True,
    'export_geotiff_coordinates': True,
    'l1r_delete_netcdf': True,
    'l2t_delete_netcdf': True,
    'l2r_delete_netcdf': True,
    'l2w_delete_netcdf': True,
}

acolite_configs.update(polygon=CAMINHO_AREA_ESTUDO_GEOJSON)

### Se esqueceu de criar alguma pasta, será criada

In [ ]:
if not exists(CAMINHO_OUTPUT_DOWNLOAD):
    makedirs(CAMINHO_OUTPUT_DOWNLOAD)

if not exists(CAMINHO_OUTPUT_EXTRAIDO):
    makedirs(CAMINHO_OUTPUT_EXTRAIDO)

if not exists(CAMINHO_OUTPUT_PROCESSADO_ACOLITE):
    makedirs(CAMINHO_OUTPUT_PROCESSADO_ACOLITE)

### Baixando as imagens SENTINEL-2 L1C

In [ ]:
# connect to the API
api = SentinelAPI(USUARIO, SENHA, 'https://apihub.copernicus.eu/apihub')

footprint = geojson_to_wkt(read_geojson(CAMINHO_AREA_ESTUDO_GEOJSON))

dt_final = date.today()

products = api.query(footprint,
                     date=(date(2016, 1, 1), dt_final),
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, 5),
                     producttype='S2MSI1C')

len(products)

In [ ]:
api.download_all(products, directory_path=CAMINHO_OUTPUT_DOWNLOAD)

### Extraindo todos os arquivos baixados

In [ ]:
for arquivo in tqdm(glob(f'{CAMINHO_OUTPUT_DOWNLOAD}/*.zip')):
    with ZipFile(arquivo, 'r') as zipado:
        zipado.extractall(CAMINHO_OUTPUT_EXTRAIDO)

### Configurando o acolite para automatizar o processamento

É necessário clonar o repositório do acolite com o comando antes de continuar:

`git clone https://github.com/acolite/acolite.git acolite_py`

In [ ]:
import sys

pwd = getcwd()

if exists(join(pwd, 'acolite')):
    os.rename('acolite', 'acolite_py')  # É necessário renomear, por acusar erro de namespace (muitas pastas com o nome acolite)

with open('acolite_py/launch_acolite.py', 'w') as launch_acolite:
    launch_acolite.write("""
def launch_acolite_py(settings):
    import sys

    try:
        import acolite as ac
    except:
        print('Could not import ACOLITE source')
        print("Error:", sys.exc_info())
        return ()

    ac.acolite.acolite_run(settings)
""")

with open('acolite_py/__init__.py', 'w') as init:
    init.write('from .launch_acolite import launch_acolite_py')
    init.write('\n')

os.chmod('acolite_py/__init__.py', stat.S_IRWXU | stat.S_IRGRP | stat.S_IROTH )
os.chmod('acolite_py/launch_acolite.py', stat.S_IRWXU | stat.S_IRGRP | stat.S_IROTH )

if join(pwd, 'acolite_py/') not in sys.path:
    sys.path.append(join(pwd, 'acolite_py/'))

if join(pwd, 'acolite_py/acolite') not in sys.path:
    sys.path.append(join(pwd, 'acolite_py/acolite'))

# if join(pwd, 'acolite/launch_acolite.py') not in sys.path:
#     sys.path.append(join(pwd, 'acolite/launch_acolite.py'))

if join(pwd, 'acolite_py/acolite/acolite/') not in sys.path:
    sys.path.append(join(pwd, 'acolite_py/acolite/acolite'))

print(sys.path)

### Processar todos os arquivos extraidos no acolite

In [ ]:
from acolite_py import launch_acolite_py

In [ ]:
PASTAS_SAFE_SENTINEL = [SAFE for SAFE in glob(f'{CAMINHO_OUTPUT_EXTRAIDO}/*', recursive=False)]

for PASTA in tqdm(PASTAS_SAFE_SENTINEL):

    acolite_configs_temporario = acolite_configs

    CAMINHO_OUTPUT_IMAGENS_PROCESSADAS = join(CAMINHO_OUTPUT_PROCESSADO_ACOLITE, basename(PASTA))

    if not exists(CAMINHO_OUTPUT_IMAGENS_PROCESSADAS):
        makedirs(CAMINHO_OUTPUT_IMAGENS_PROCESSADAS)

        acolite_configs_temporario.update(inputfile=PASTA, output=CAMINHO_OUTPUT_IMAGENS_PROCESSADAS)

        launch_acolite_py(settings=acolite_configs_temporario)

        clear_output()
    else:
        print("Já fui processado, pulando para o próximo!!")